# **Install Library**

In [ ]:
%pip uninstall -y google-auth-oauthlib google-auth-httplib2 google-auth google-api-python-client
%pip install --upgrade google-auth-oauthlib==0.4.1 google-auth-httplib2 google-auth==2.27.0 google-api-python-client

# **Import Library**

In [ ]:
import os
import base64
import re
import pandas as pd

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# **Utama**

In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow

flow = InstalledAppFlow.from_client_secrets_file(
    "client_secret.json", scopes=["https://www.googleapis.com/auth/gmail.readonly"]
)

credentials = flow.run_console()
gmail_service = build("gmail", "v1", credentials=credentials)

# Clear the output for safety
from IPython.display import clear_output
clear_output()

In [ ]:
import re

def extract_domain(email):
  """Extracts the domain name from an email address.

  Args:
    email: The email address to extract the domain from.

  Returns:
    The domain name, or None if the email address is invalid.
  """
  match = re.search(r"@[\w.\-]+", email)
  if match:
    return match.group().lstrip('@')
  return None

In [ ]:
# Extract titles with the Gmail API
extract_limit = 10000
extracted_count = 0
titles = []

next_page_token = None

while(extracted_count < extract_limit):
  results = gmail_service.users().messages().list(userId='me', maxResults=500).execute()
  messages = results.get('messages', [])
  result_size_estimate = results.get('resultSizeEstimate', 0)
  next_page_token = results.get('nextPageToken', None)

  if len(messages) == 0:
    break

  # The extracted title should contain: {actual title} {sender domain}, to hopefully help with automatic tagging.
  for message in messages:
    # Fetching metadata, including headers.
    msg = gmail_service.users().messages().get(userId='me', id=message['id'], format='metadata').execute()
    headers = msg['payload']['headers']

    # Extracting subject from headers.
    subject = next((header['value'] for header in headers if header['name'] == 'Subject'), '')
    from_email = next((header['value'] for header in headers if header['name'] == 'From'), '')

    # Extract domain
    from_domain = extract_domain(from_email)

    titles.append(f"{subject} from:{from_domain}")

  if next_page_token == None:
    break

  extracted_count += len(messages)
  print(f"Extracted {extracted_count}/{extract_limit} emails so far...")

titles[:10]

In [ ]:
# Turn into a CSV and download
df = pd.DataFrame(titles, columns=['title'])
df.to_csv('email-titles.csv', index=False)

from google.colab import files
files.download('email-titles.csv')